# Error Analysis
This is a short example for anaylzing the trained model's prediction errors.

### Model Inference

In [ ]:
!git clone https://github.com/AMLS-PRG/AtomML-Course

In [ ]:
!pip install numpy matplotlib deepmd-kit dpdata

In [ ]:
from deepmd.infer import DeepPot
import numpy as np
from matplotlib import pyplot as plt
import dpdata
from deepmd.common import expand_sys_str
from deepmd import DeepPotential

In [ ]:
model = DeepPotential("AtomML-Course/module-5/02-Training-machine-learning-potential-MLP/Trained-MLPs/frozen_model_1_compressed.pb")
all_systems = expand_sys_str("AtomML-Course/module-5/01-Preparing-training-data/dataset")
E_pred, F_pred, E_true, F_true = [], [], [], []
for system in all_systems:
    data = dpdata.LabeledSystem(system, fmt='deepmd/npy')
    energy, force, virial = model.eval(data['coords'], data['cells'], data['atom_types'])
    E_pred.append(energy[:,0])
    F_pred.append(force)
    E_true.append(data['energies'])
    F_true.append(data['forces'])
E_pred, F_pred = np.concatenate(E_pred), np.concatenate(F_pred)
E_true, F_true = np.concatenate(E_true), np.concatenate(F_true)

### RMSE

In [ ]:
print('Energy RMSE (eV):      %.4e' % ((E_pred - E_true)**2).mean()**0.5)
print('Energy RMSE/atom (eV): %.4e' % (((E_pred - E_true)**2).mean()**0.5 / data['coords'].shape[1]))
print('Force RMSE (eV/A):     %.4e' % ((F_pred - F_true)**2).mean()**0.5)

### Parity Plots

In [ ]:
plt.plot(F_true.flatten(), F_true.flatten())
plt.plot(F_pred.flatten(), F_true.flatten(), '.')

### Error Distribution

In [ ]:
histogram = plt.hist((F_pred - F_true).flatten(), bins=300, density=True)